In [2]:
import pandas
import workdays
from os import error
from typing import Optional
import config
import psycopg2
import pandas as pd
import workdays
import datetime
import numpy as np
holidays_br = workdays.load_holidays("BR")
holidays_b3 = workdays.load_holidays("B3")
dt = datetime.date.today()
dt_1 = workdays.workday(dt, -1, holidays_b3)

def get_alugueis(dt_1, dt_liq):
    db_conn_risk = psycopg2.connect(
        host=config.DB_RISK_HOST,
        dbname=config.DB_RISK_NAME,
        user=config.DB_RISK_USER,
        password=config.DB_RISK_PASS,
    )
    query = f"SELECT st_alugcustcorr.contrato, registro, corretora, st_alugcustcorr.cliente as fundo,reversor, codigo, \
    vencimento, taxa, (avg(qtde)+ case when sum(qteliq) is null then '0' else  sum(qteliq) end) as quantidade, avg(cotliq) as preco_init, \
    negeletr, (case when sum(qtde) < 0  then 'D' else  'T' end) as Tipo \
    from st_alugcustcorr left join st_alug_devolucao on st_alugcustcorr.cliente=st_alug_devolucao.cliente and \
    st_alugcustcorr.contrato=st_alug_devolucao.contrato and dataliq='{dt_liq.strftime('%Y-%m-%d')}' \
    where data='{dt_1.strftime('%Y-%m-%d')}' and st_alugcustcorr.cliente<>''  AND st_alugcustcorr.cliente like 'KAPITALO KAPPA MASTER FIM%' \
    group by st_alugcustcorr.contrato,registro, corretora,st_alugcustcorr.cliente,reversor,codigo, vencimento, taxa, st_alugcustcorr.negeletr  \
    HAVING (avg(qtde)+ case when sum(qteliq) is null then '0' else  sum(qteliq) end)<>0  \
    order by codigo,vencimento,st_alugcustcorr.cliente,contrato"
    df = pd.read_sql(query, db_conn_risk)
    db_conn_risk.close()
    return df



alug=get_alugueis(dt_1,dt)
alug['taxa']=alug['taxa']*100

alug=alug[['codigo','contrato','quantidade','taxa','corretora']]
doador=alug[alug['quantidade']<0]

doador=doador[['codigo','quantidade','taxa']].groupby('codigo').agg({'quantidade':sum,'taxa':np.mean}).reset_index()
doador
# mm=df[df['corretora']=='BTG PACTUAL CTVM S/A'][['codigo','quantidade','taxa']].groupby('codigo').agg({'quantidade':sum,'taxa':np.mean})



,codigo,quantidade,taxa
0,BOVA11,-1804291.0,5.044909
1,BOVV11,-97890.0,5.811379
2,BPAN4,-55553.0,1.102500
3,CBAV3,-2000.0,0.090000
4,CCRO3,-12333.0,4.450000
5,CYRE3,-2469.0,1.550000
6,IRBR3,-34800.0,80.000000
7,MGLU3,-1744325.0,8.215333
8,MOVI3,-1800.0,2.640000
9,SIMH3,-2400.0,0.115000


In [3]:
def get_taxa(ticker_name, pos):
    db_conn_risk = psycopg2.connect(
    host=config.DB_RISK_HOST,
    dbname=config.DB_RISK_NAME,
    user=config.DB_RISK_USER,
    password=config.DB_RISK_PASS,
    )

    try:
        query = f"""SELECT rptdt, tckrsymb, sctyid, sctysrc, mktidrcd, isin, asst, qtyctrctsday, qtyshrday, valctrctsday, dnrminrate, dnravrgrate, dnrmaxrate, takrminrate, takravrgrate, takrmaxrate, mkt, mktnm, datasts \
            FROM b3up2data.equities_assetloanfilev2 \
            where   tckrsymb = '{ticker_name}' and mktnm = 'Balcao' order by rptdt desc
            limit 1;
            """
        df = pd.read_sql(query, db_conn_risk)
        tx = float(df.iloc[pos]["takravrgrate"])
    
    except:
        tx=0
        # query = f"""SELECT ticker, ts, quantity, rate, id
		# 	FROM public.aluguel_b3 where ts>='{dt_1.strftime("%Y-%m-%d")}';"""
        # df = pd.read_sql(query, db_conn_test)
        # df["s"] = df["quantity"] * df["rate"]
        # df = df.groupby("ticker").sum()
        # df["avg_rate"] = df["s"] / df["quantity"]
        # tx = float(df.loc[ticker_name, "avg_rate"])
    
    db_conn_risk.close()
    return tx


In [4]:
def get_last_pric(ticker,dt):
    db_conn_risk = psycopg2.connect(
    host=config.DB_RISK_HOST,
    dbname=config.DB_RISK_NAME,
    user=config.DB_RISK_USER,
    password=config.DB_RISK_PASS,
    )
    query=f"SELECT lastpric FROM b3up2data.equities_eodpricefile where tckrsymb='{ticker}' and rptdt='{dt.strftime('%Y-%m-%d')}' "
    df = pd.read_sql(query, db_conn_risk)
    try:
        tx = float(df.iloc[0]["lastpric"])
    
    except:
        tx=0
    db_conn_risk.close()
    return tx

In [5]:
tomador=alug[alug['quantidade']>0]

tomador=tomador[['codigo','quantidade','taxa','corretora']].groupby(['codigo']).agg({'quantidade':sum,'taxa':np.mean}).reset_index()

tomador
# subs=tomador[tomador['corretora']=='BTG PACTUAL CTVM S/A']

# subs['id']='MM'

# subs=subs.rename(columns={'quantidade':'tomado'})

# subs

,codigo,quantidade,taxa
0,ABEV3,706740.0,0.637714
1,ALPA4,10914.0,0.780000
2,ALSO3,3800.0,1.550000
3,AMER3,34397.0,0.882857
4,ASAI3,42439.0,0.150000
...,...,...,...
97,VIIA3,195702.0,3.570000
98,VIVA3,2700.0,0.250000
99,VIVT3,23166.0,1.840000
100,WEGE3,134345.0,0.093889


In [6]:
## Analisar composição da carteira 
conn = psycopg2.connect(
    host=config.DB_TESTE_HOST,
    dbname=config.DB_TESTE_NAME,
    user=config.DB_TESTE_USER,
    password=config.DB_TESTE_PASS,
)
query=f"select str_serie as codigo,dbl_lote from tbl_carteira1 where dte_data='{dt_1.strftime('%Y-%m-%d')}' and str_mesa='Kapitalo 11.1' and str_mercado='Acao'"

carteira=pd.read_sql(query,conn)

carteira=carteira.groupby('codigo').sum().reset_index()
carteira['side']=carteira['dbl_lote'].apply(lambda x: "Long" if x>0 else "Short" )

carteira['codigo']=carteira['codigo'].apply(lambda x: x.replace(' BZ EQUITY',''))
carteira

side=carteira[['codigo','side']]
side

,codigo,side
0,ABEV US EQUITY,Long
1,ABEV3,Short
2,AERI3,Long
3,ALPA4,Short
4,ALSO3,Short
...,...,...
111,VIVA3,Short
112,VIVT3,Short
113,WEGE3,Short
114,WIZS3,Long


In [7]:
##Carteira MM 
    
conn = psycopg2.connect(
    host=config.DB_TESTE_HOST,
    dbname=config.DB_TESTE_NAME,
    user=config.DB_TESTE_USER,
    password=config.DB_TESTE_PASS,
)
query=f"select str_serie as codigo ,dbl_lote,str_estrategia from tbl_carteira1 where dte_data='{dt_1.strftime('%Y-%m-%d')}' and str_mesa='Kapitalo 11.1' and str_estrategia='MM' and str_mercado ='Acao'"


mm=pd.read_sql(query,conn)


mm=mm[['codigo','dbl_lote','str_estrategia']]
mm['codigo']=mm['codigo'].apply(lambda x: x.replace(' BZ EQUITY',''))

mm=mm.merge(side,on='codigo',how='left')
## Interna verdadeira quando operação é subsidiada por outra estrategia
mm['intern']=mm.apply(lambda row: True if (row['dbl_lote']<0 and row['side']=='Long') else True if(row['dbl_lote']>0 and row['side']=='Short') else False ,axis=1)

mm


# aloc=df.merge(subs[['codigo','tomado','taxa']],on=['codigo'],how='left').fillna(0)

# aloc_sub=aloc[aloc['tomado']!=0]
# aloc_sub=aloc_sub.reset_index(drop=True)
# aloc_sub['id']='MM - Subsidiado'

# aloc_sub['net']=aloc_sub.apply(lambda row: row['tomado']+row['dbl_lote'],axis=1)
# display(aloc_sub)

# for index,row in aloc_sub.iterrows():
#     if row['net']<0:
#         aloc_sub.loc[index,'dbl_lote']=-row['tomado']
#         aloc_sub=aloc_sub.append({'codigo':row['codigo'],'dbl_lote':row['net'],'tomado':0,'id':'MM - Mercado','net':-1,'taxa':-1},ignore_index=True)
        
# aloc_sub['net']=aloc_sub.apply(lambda row: row['tomado']+row['dbl_lote'],axis=1)       
# aloc_sub
mm_real=mm[mm['intern']==False]
mm_intern=mm[mm['intern']==True]

mm_real

,codigo,dbl_lote,str_estrategia,side,intern
0,ABEV3,-178900.0,MM,Short,False
1,AERI3,4600.0,MM,Long,False
2,ALSO3,-1900.0,MM,Short,False
3,AMER3,-3700.0,MM,Short,False
4,AZUL4,16200.0,MM,Long,False
5,B3SA3,-34700.0,MM,Short,False
6,BBAS3,-30500.0,MM,Short,False
8,BBSE3,-14700.0,MM,Short,False
9,BOVA11,31277.0,MM,Long,False
10,BPAC11,-19000.0,MM,Short,False


In [8]:
##Carteira Cash

conn = psycopg2.connect(
    host=config.DB_TESTE_HOST,
    dbname=config.DB_TESTE_NAME,
    user=config.DB_TESTE_USER,
    password=config.DB_TESTE_PASS,
)
query=f"select str_serie as codigo ,dbl_lote,str_estrategia from tbl_carteira1 where dte_data='{dt_1.strftime('%Y-%m-%d')}' and str_mesa='Kapitalo 11.1' and str_estrategia like '%CashCarry%' and str_mercado ='Acao'"


cash=pd.read_sql(query,conn)


cash=cash[['codigo','dbl_lote','str_estrategia']]
cash['codigo']=cash['codigo'].apply(lambda x: x.replace(' BZ EQUITY',''))

cash=cash.merge(side,on='codigo',how='left')
## Interna verdadeira quando operação é subsidiada por outra estrategia
cash['intern']=cash.apply(lambda row: True if (row['dbl_lote']<0 and row['side']=='Long') else True if(row['dbl_lote']>0 and row['side']=='Short') else False ,axis=1)

cash_intern = cash[cash['intern']==True]
cash_real = cash[cash['intern']==False]
cash_intern

,codigo,dbl_lote,str_estrategia,side,intern
4,AZUL4,-9632.0,CashCarry,Long,True
12,BPAN4,-9993.0,CashCarry,Long,True
17,CASH3,-16061.0,CashCarry,Long,True
18,CCRO3,-32567.0,CashCarry,Long,True
21,CMIN3,-32866.0,CashCarry,Long,True
25,CRFB3,-12062.0,CashCarry,Long,True
29,CYRE3,-8231.0,CashCarry,Long,True
31,ECOR3,-9968.0,CashCarry,Long,True
34,ELET6,-7030.0,CashCarry,Long,True
36,ENBR3,-6812.0,CashCarry,Long,True


In [9]:
## Carteira Box

conn = psycopg2.connect(
    host=config.DB_TESTE_HOST,
    dbname=config.DB_TESTE_NAME,
    user=config.DB_TESTE_USER,
    password=config.DB_TESTE_PASS,
)
query2=f"select str_serie as codigo ,dbl_lote,str_estrategia from tbl_carteira1 where dte_data='{dt_1.strftime('%Y-%m-%d')}' and str_mesa='Kapitalo 11.1' and str_estrategia = 'Box_3pontas' and str_mercado ='Acao'"


box=pd.read_sql(query2,conn)

box=box[['codigo','dbl_lote','str_estrategia']]
box['codigo']=box['codigo'].apply(lambda x: x.replace(' BZ EQUITY',''))

box=box.merge(side,on='codigo',how='left')
## Interna verdadeira quando operação é subsidiada por outra estrategia
box['intern']=box.apply(lambda row: True if (row['dbl_lote']<0 and row['side']=='Long') else True if(row['dbl_lote']>0 and row['side']=='Short') else False ,axis=1)
box_intern = box[box['intern']==True]
box_real = box[box['intern']==False]
# box[box['intern']==True]
# box['borrow sub']=box.apply(lambda x: x['dbl_lote'] if x['intern']==True else 0,axis=1)
# box

In [10]:
box[box['intern']==False]

,codigo,dbl_lote,str_estrategia,side,intern
2,CASH3,1100.0,Box_3pontas,Long,False
3,CCRO3,68300.0,Box_3pontas,Long,False
5,IRBR3,-196600.0,Box_3pontas,Short,False
7,MGLU3,2304900.0,Box_3pontas,Long,False


In [10]:
## Adicionar boletas internas e ALocar alugueis reais em posições convergentes

In [11]:
intern=pd.concat([mm_intern ,cash_intern, box_intern ])
intern['doado'] = intern.apply(lambda row: row['dbl_lote'] if row['side']=='Short' else 0,axis=1 )
intern['tomado'] = intern.apply(lambda row: row['dbl_lote'] if row['side']=='Long' else 0,axis=1) 
intern

# real=pd.concat([cash[cash['intern']==False] ,mm[mm['intern']==False],   box[box['intern']==False] ])
# real


,codigo,dbl_lote,str_estrategia,side,intern,doado,tomado
7,BBDC4,286656.0,MM,Short,True,286656.0,0.0
14,CASH3,-49000.0,MM,Long,True,0.0,-49000.0
21,CVCB3,5600.0,MM,Short,True,5600.0,0.0
24,ECOR3,-4500.0,MM,Long,True,0.0,-4500.0
35,IRBR3,180000.0,MM,Short,True,180000.0,0.0
36,ITSA4,45000.0,MM,Short,True,45000.0,0.0
44,MGLU3,-771100.0,MM,Long,True,0.0,-771100.0
46,MRFG3,100.0,MM,Short,True,100.0,0.0
47,MRVE3,-4800.0,MM,Long,True,0.0,-4800.0
48,PETR4,28000.0,MM,Short,True,28000.0,0.0


In [12]:
real=pd.concat([mm_real ,cash_real, box_real ])

# real_tomado=real[real['side']=='Short']

# real_tomado

In [13]:
total=real.groupby("codigo").sum()[['dbl_lote']]

In [14]:

df_ctosaluguel = get_alugueis(dt_1, dt)
display(df_ctosaluguel)
df_ctosaluguel["value"] = df_ctosaluguel["taxa"] * df_ctosaluguel["quantidade"]
df_ctosaluguel["value_price"] = df_ctosaluguel["preco_init"] * df_ctosaluguel["quantidade"]
df_doado = df_ctosaluguel[df_ctosaluguel["quantidade"] < 0]
df_doado = df_doado.groupby("codigo", as_index=False).agg(
    {"quantidade": sum, "value": sum,"value_price":sum}
    )
df_doado["taxa_doado"] = round(df_doado["value"] * 100 / df_doado["quantidade"], 2)
df_doado['preco'] = round(df_doado["value_price"]/ df_doado["quantidade"], 2)
df_tomado = df_ctosaluguel[df_ctosaluguel["quantidade"] > 0]
df_tomado = df_tomado.groupby("codigo", as_index=False).agg(
    {"quantidade": sum, "value": sum,"value_price":sum}
    )
df_tomado["taxa_tomado"] = round(df_tomado["value"] * 100 / df_tomado["quantidade"], 2)
df_tomado["preco"] = round(df_tomado["value_price"] / df_tomado["quantidade"], 2)




,contrato,registro,corretora,fundo,reversor,codigo,vencimento,taxa,quantidade,preco_init,negeletr,tipo
0,2022080300359264240001-2,2022-08-03,MERRILL LYNCH S/A CTVM,KAPITALO KAPPA MASTER FIM,TD,ABEV3,2022-09-13,0.0067,95063.0,14.67,False,T
1,2022080300367426720001-1,2022-08-03,ITAU CV S/A,KAPITALO KAPPA MASTER FIM,TD,ABEV3,2022-09-13,0.0067,643.0,14.67,False,T
2,2022080500368359500001-1,2022-08-05,MERRILL LYNCH S/A CTVM,KAPITALO KAPPA MASTER FIM,TD,ABEV3,2022-09-15,0.0067,40000.0,14.56,False,T
3,2022080500368380410001-1,2022-08-05,MERRILL LYNCH S/A CTVM,KAPITALO KAPPA MASTER FIM,TD,ABEV3,2022-09-15,0.0067,4066.0,14.56,False,T
4,2022080500368422730001-1,2022-08-05,MERRILL LYNCH S/A CTVM,KAPITALO KAPPA MASTER FIM,TD,ABEV3,2022-09-15,0.0067,38571.0,14.56,False,T
...,...,...,...,...,...,...,...,...,...,...,...,...
1857,2022082200373360830001-1,2022-08-22,ITAU CV S/A,KAPITALO KAPPA MASTER FIM,TD,YDUQ3,2022-09-30,0.0552,100.0,12.52,False,T
1858,2022082300355814160001-2,2022-08-23,MERRILL LYNCH S/A CTVM,KAPITALO KAPPA MASTER FIM,TD,YDUQ3,2022-10-03,0.0619,529.0,12.13,False,T
1859,2022082300363305360001-2,2022-08-23,MERRILL LYNCH S/A CTVM,KAPITALO KAPPA MASTER FIM,TD,YDUQ3,2022-10-03,0.0619,7220.0,12.13,False,T
1860,2022082400373995070001-1,2022-08-24,MERRILL LYNCH S/A CTVM,KAPITALO KAPPA MASTER FIM,TD,YDUQ3,2022-10-04,0.0608,200.0,12.27,False,T


In [15]:
df_tomado=df_tomado[['codigo','quantidade','taxa_tomado','preco']].rename(columns={'quantidade':'tomado'})
df_doado=df_doado[['codigo','quantidade','taxa_doado','preco']].rename(columns={'quantidade':'doado'})
df_tomado

,codigo,tomado,taxa_tomado,preco
0,ABEV3,706740.0,0.58,15.10
1,ALPA4,10914.0,0.78,18.86
2,ALSO3,3800.0,1.55,18.53
3,AMER3,34397.0,0.88,14.75
4,ASAI3,42439.0,0.15,17.74
...,...,...,...,...
97,VIIA3,195702.0,4.19,3.38
98,VIVA3,2700.0,0.25,22.02
99,VIVT3,23166.0,1.84,44.48
100,WEGE3,134345.0,0.08,28.98


In [16]:
real=real.merge(df_tomado,on='codigo',how='left').fillna(0)

In [17]:
real=real.merge(df_doado,on='codigo',how='left').fillna(0)

In [18]:
real[real['codigo']=='ABEV3']

,codigo,dbl_lote,str_estrategia,side,intern,tomado,taxa_tomado,preco_x,doado,taxa_doado,preco_y
0,ABEV3,-178900.0,MM,Short,False,706740.0,0.58,15.1,0.0,0.0,0.0
57,ABEV3,-128800.0,CashCarry,Short,False,706740.0,0.58,15.1,0.0,0.0,0.0


In [19]:
intern["taxa"] = intern["codigo"].apply(lambda x: get_taxa(ticker_name=x, pos=0))
intern['str_estrategia'] = intern['str_estrategia'].apply(lambda x: x+'\Interna')
intern_doado=intern[['codigo','doado','taxa']].groupby('codigo').agg({'doado':sum,'taxa':np.mean})
intern_tomado=intern[['codigo','tomado','taxa']].groupby('codigo').agg({'tomado':sum,'taxa':np.mean})


In [20]:
intern_doado=intern_doado[intern_doado['doado']!=0]

intern_tomado=intern_tomado[intern_tomado['tomado']!=0]

In [21]:
intern_doado

,doado,taxa
codigo,,
ABEV3,188600.0,0.56
BBDC4,286656.0,0.03
CVCB3,5700.0,5.63
IRBR3,180000.0,93.97
ITSA4,45000.0,0.24
LWSA3,114900.0,12.61
MRFG3,100.0,0.51
PETR4,28000.0,0.03
SBSP3,1700.0,0.09


In [22]:
intern_tomado


,tomado,taxa
codigo,,
AZUL4,-9632.0,1.44
BOVA11,-308316.0,3.42
BPAN4,-9993.0,1.33
CASH3,-65061.0,3.03
CCRO3,-32567.0,4.44
CMIN3,-32866.0,0.16
CRFB3,-12062.0,2.95
CYRE3,-8231.0,1.55
ECOR3,-14468.0,0.44


In [23]:
## Destribuindo Internas 

for index,row in real.iterrows():
    if row['side']=='Short':
        if row['codigo'] in intern_doado.index.tolist():
            real.loc[index,'tomado']+=intern_doado.loc[row['codigo'],'doado']
            real.loc[index,'tomado']=round(real.loc[index,'tomado'] * (row['dbl_lote']/total.loc[row['codigo'],'dbl_lote'] ),0)
            real.loc[index,'taxa_tomado']=round( (row['dbl_lote']*row['taxa_tomado']+intern_doado.loc[row['codigo'],'taxa']*intern_doado.loc[row['codigo'],'doado'])/(row['dbl_lote']+intern_doado.loc[row['codigo'],'doado']),2)
        else:
            real.loc[index,'tomado']=round(real.loc[index,'tomado'] * (row['dbl_lote']/total.loc[row['codigo'],'dbl_lote'] ),0)
    else:
        if row['codigo'] in intern_tomado.index.tolist():    
            real.loc[index,'doado']-=intern_tomado.loc[row['codigo'],'tomado']
            real.loc[index,'doado']=round(real.loc[index,'doado'] * (row['dbl_lote']/total.loc[row['codigo'],'dbl_lote'] ),0)
            real.loc[index,'taxa_doado']=round( (row['dbl_lote']*row['taxa_doado']+intern_tomado.loc[row['codigo'],'taxa']*intern_tomado.loc[row['codigo'],'tomado'])/(row['dbl_lote']+intern_tomado.loc[row['codigo'],'tomado']),2)
        else:
            real.loc[index,'doado']=round(real.loc[index,'doado'] * (row['dbl_lote']/total.loc[row['codigo'],'dbl_lote'] ),0)

In [24]:
real[real['codigo']=='VIVT3']

,codigo,dbl_lote,str_estrategia,side,intern,tomado,taxa_tomado,preco_x,doado,taxa_doado,preco_y
126,VIVT3,-12066.0,CashCarry,Short,False,23166.0,1.84,44.48,0.0,0.0,0.0


In [25]:
# .to_clipboard(excel='True')
complete=pd.concat([real,intern]).fillna(0)
complete['taxa_tomado']=complete.apply(lambda row: row['taxa']  if (row['tomado']!=0 and row['taxa']!=0) else  row['taxa_tomado'] ,axis=1)
complete['taxa_doado']=complete.apply(lambda row: row['taxa']  if (row['doado']!=0 and row['taxa']!=0) else  row['taxa_doado'],axis=1)

complete['preco']=complete.apply(lambda row: row['preco_x']  if (row['preco_x']!=0) else row['preco_y'] if (row['preco_y']!=0) else get_last_pric(row['codigo'],dt_1) ,axis=1)

In [26]:

complete['fin doado']=complete.apply(lambda row: abs(row['doado']*row['preco'] * ((( 1+ row['taxa_doado']/100) ** (1/252) ) -1 )) if row['taxa_doado']!=0 else 0 ,axis=1)


complete['fin tomado']=complete.apply(lambda row: -abs(row['tomado']*row['preco'] *(((1+row['taxa_tomado']/100) ** (1/252) ) -1))   if row['taxa_tomado']!=0 else 0 ,axis=1)

                                      
complete['financeiro']=complete['fin doado']+complete['fin tomado']

In [27]:
complete[complete['str_estrategia']=='CashCarry'].sort_values('financeiro',ascending=True)

,codigo,dbl_lote,str_estrategia,side,intern,tomado,taxa_tomado,preco_x,doado,taxa_doado,preco_y,taxa,preco,fin doado,fin tomado,financeiro
97,ITUB4,-140714.0,CashCarry,Short,False,250202.0,0.90,26.50,0.0,0.0,0.0,0.0,26.50,0.0,-235.743271,-235.743271
106,PCAR3,-4653.0,CashCarry,Short,False,756141.0,0.34,16.61,0.0,0.0,0.0,0.0,16.61,0.0,-169.167318,-169.167318
79,CVCB3,-8070.0,CashCarry,Short,False,21700.0,-13.20,11.32,0.0,0.0,0.0,0.0,11.32,0.0,-137.954264,-137.954264
57,ABEV3,-128800.0,CashCarry,Short,False,374780.0,0.52,15.10,0.0,0.0,0.0,0.0,15.10,0.0,-116.475317,-116.475317
113,SANB11,-10609.0,CashCarry,Short,False,18809.0,4.72,29.77,0.0,0.0,0.0,0.0,29.77,0.0,-102.487862,-102.487862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,BRAP4,-7384.0,CashCarry,Short,False,10628.0,0.08,23.35,0.0,0.0,0.0,0.0,23.35,0.0,-0.787508,-0.787508
93,IGTI11,-5114.0,CashCarry,Short,False,9614.0,0.07,20.65,0.0,0.0,0.0,0.0,20.65,0.0,-0.551278,-0.551278
83,EMBR3,-21538.0,CashCarry,Short,False,39138.0,0.02,14.03,0.0,0.0,0.0,0.0,14.03,0.0,-0.435755,-0.435755
120,TCSA3,-3.0,CashCarry,Short,False,3.0,3.75,3.05,0.0,0.0,0.0,0.0,3.05,0.0,-0.001337,-0.001337


In [28]:
complete

,codigo,dbl_lote,str_estrategia,side,intern,tomado,taxa_tomado,preco_x,doado,taxa_doado,preco_y,taxa,preco,fin doado,fin tomado,financeiro
0,ABEV3,-178900.0,MM,Short,False,520560.0,0.19,15.10,0.0,0.00,0.0,0.00,15.10,0.000000,-59.209335,-59.209335
1,AERI3,4600.0,MM,Long,False,0.0,0.00,0.00,0.0,0.00,0.0,0.00,2.33,0.000000,0.000000,0.000000
2,ALSO3,-1900.0,MM,Short,False,3800.0,1.55,18.53,0.0,0.00,0.0,0.00,18.53,0.000000,-4.297928,-4.297928
3,AMER3,-3700.0,MM,Short,False,5976.0,0.88,14.75,0.0,0.00,0.0,0.00,14.75,0.000000,-3.064703,-3.064703
4,AZUL4,16200.0,MM,Long,False,1632.0,2.30,16.22,9632.0,-2.11,0.0,0.00,16.22,13.220670,-2.388750,10.831920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,VIIA3,-46902.0,CashCarry\Interna,Long,True,-46902.0,4.23,0.00,0.0,0.00,0.0,4.23,3.20,0.000000,-24.676834,-24.676834
0,ABEV3,188600.0,Box_3pontas\Interna,Short,True,0.0,0.00,0.00,188600.0,0.56,0.0,0.56,15.43,64.489159,0.000000,64.489159
1,BOVA11,-308316.0,Box_3pontas\Interna,Long,True,-308316.0,3.42,0.00,0.0,0.00,0.0,3.42,106.59,0.000000,-4385.754705,-4385.754705
4,CVCB3,100.0,Box_3pontas\Interna,Short,True,0.0,0.00,0.00,100.0,5.63,0.0,5.63,7.80,0.169552,0.000000,0.169552


In [29]:
display(complete.groupby('str_estrategia').agg({'financeiro':sum}))
print("Financeiro dia aproximado: ",round(complete['financeiro'].sum(),2))

,financeiro
str_estrategia,
Box_3pontas,-4964.005710
Box_3pontas\Interna,-3805.475475
CashCarry,-2007.966490
CashCarry5,30177.029418
CashCarry\Interna,-479.482798
MM,-939.088651
MM\Interna,-364.868290


Financeiro dia aproximado:  17616.14


In [30]:
complete['financeiro'].sum()

14916.602280883137

In [31]:
complete.groupby('codigo').agg({'dbl_lote':sum,'tomado':sum,'taxa_tomado':np.mean,'doado':sum,'taxa_doado':np.mean,'preco':np.mean}).fillna(0)

,dbl_lote,tomado,taxa_tomado,doado,taxa_doado,preco
codigo,,,,,,
ABEV3,-427127.0,398827.0,0.28,0.0,0.00,13.610
AERI3,4600.0,0.0,0.00,-4600.0,5.29,2.600
ALPA4,-12329.0,11029.0,2.11,0.0,0.00,19.390
AMER3,-40226.0,36326.0,1.00,0.0,0.00,14.400
ARZZ3,5300.0,0.0,0.00,-126.0,3.70,69.350
...,...,...,...,...,...,...
VIVA3,-2700.0,2700.0,0.20,0.0,0.00,22.700
VIVT3,-25426.0,22726.0,3.05,0.0,0.00,46.600
WEGE3,-134751.0,128727.0,0.10,0.0,0.00,25.910


Análise Retroativa

In [32]:
query= "select dte_data,dbl_ganho from tbl_gerencialvalorizacao where dte_data>'2022-01-01' and str_mesa='Kapitalo 11.1' and str_fundo like '%KAPPA%' and str_estrategia='Bolsa' and str_mercado='Aluguel de Acao'"

df=pd.read_sql(query,conn)

In [33]:

fig=px.plot(df)

NameError: name 'px' is not defined

In [ ]:
df['dbl_ganho'].sum()